In [1]:
import os
os.chdir("/Users/jiacheng/Library/CloudStorage/GoogleDrive-huan1660@umn.edu/.shortcut-targets-by-id/1S-w1mPkATg55q7rjjLiLX--1rcqAIftF/PR_Scrape")


# 1. Clean the Metadata and the STM Topic

In [2]:
import pandas as pd

# Read JSONL file
df = pd.read_json("02_Output_CSA_CSR/02_DTA_PR_w_Topic.jsonl", lines=True)

df

,url,journal,year,title,full_text,citation,alternative.id,created,deposited,doi,...,Topic86,Topic87,Topic88,Topic89,Topic90,Topic91,Topic92,Topic93,CSR,CSA
0,http://dx.doi.org/10.1016/j.pubrev.2003.09.003,Public Relations Review,2004,Exploring public relations in Croatia through ...,1 Introduction Public relations practices vary...,17,S0363811104000219,2004-05-01,2020-04-02,10.1016/j.pubrev.2003.09.003,...,0.0099,0.0014,0.0001,0.0013,0.0002,0.0079,0.0151,0.0065,0.0018,0.00000
1,http://dx.doi.org/10.1016/j.pubrev.2003.11.001,Public Relations Review,2004,Cross-national conflict shifting: a global pub...,1 Introduction A maturation process of globali...,13,S0363811103000948,2004-01-27,2020-03-29,10.1016/j.pubrev.2003.11.001,...,0.0002,0.0006,0.0258,0.0312,0.0150,0.0000,0.0007,0.0047,0.0835,0.00020
2,http://dx.doi.org/10.1016/j.pubrev.2003.11.002,Public Relations Review,2004,Strategic public diplomacy and local press: ho...,As strategic public diplomacy events head of s...,32,S036381110300095X,2004-01-23,2019-02-15,10.1016/j.pubrev.2003.11.002,...,0.0009,0.0002,0.0007,0.0002,0.0003,0.0000,0.0001,0.0443,0.0017,0.00000
3,http://dx.doi.org/10.1016/j.pubrev.2003.11.003,Public Relations Review,2004,International agenda-building in cyberspace: a...,A 1970 New York Times magazine article argued ...,16,S0363811103000961,2004-01-22,2019-02-15,10.1016/j.pubrev.2003.11.003,...,0.0022,0.0001,0.0002,0.0008,0.0011,0.0008,0.0052,0.0008,0.0014,0.00000
4,http://dx.doi.org/10.1016/j.pubrev.2003.11.004,Public Relations Review,2004,American Airlines’ use of mediated employee ch...,1 Introduction The September 11 2001 attacks h...,13,S0363811103000973,2004-01-22,2020-03-29,10.1016/j.pubrev.2003.11.004,...,0.0001,0.0000,0.0083,0.0007,0.0040,0.0002,0.0564,0.0012,0.0094,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4123,https://doi.org/10.1207/s1532754xjprr1803_4,Journal of Public Relations Research,2006,First- and Second-Level Agenda-Building and Ag...,In today s modern election campaigns the role ...,128,10.1207/s1532754xjprr1803_4,2006-08-25,2024-02-06,10.1207/s1532754xjprr1803_4,...,0.0001,0.0004,0.0001,0.0001,0.0002,0.0001,0.0002,0.0004,0.0004,0.00010
4124,https://doi.org/10.1207/s1532754xjprr1804_1,Journal of Public Relations Research,2006,Toward Public Relations Theory-Based Study of ...,There was a time when a world of corporations ...,54,10.1207/s1532754xjprr1804_1,2006-09-07,2020-04-18,10.1207/s1532754xjprr1804_1,...,0.0005,0.0296,0.0001,0.0134,0.0003,0.0001,0.0702,0.0002,0.0056,0.00001
4125,https://doi.org/10.1207/s1532754xjprr1804_2,Journal of Public Relations Research,2006,How Activist Groups Use Websites in Media Rela...,Public relations scholarship has traditionally...,96,10.1207/s1532754xjprr1804_2,2006-09-07,2019-04-21,10.1207/s1532754xjprr1804_2,...,0.0000,0.0003,0.0003,0.0004,0.0016,0.0001,0.0008,0.0003,0.0118,0.00000
4126,https://doi.org/10.1207/s1532754xjprr1804_3,Journal of Public Relations Research,2006,Responding to Activism: An Experimental Analys...,A review of literature indicates that minimal ...,29,10.1207/s1532754xjprr1804_3,2006-09-07,2019-01-13,10.1207/s1532754xjprr1804_3,...,0.0000,0.2571,0.0836,0.0005,0.0030,0.0004,0.0002,0.0002,0.0850,0.00020


In [3]:
from gensim.models import Word2Vec
import numpy as np
from tqdm import tqdm


# Load full model
model = Word2Vec.load("01_Scripts_CSA_CSR/07_PREMBD/models/word2vec_5gram_full.model").wv

# Identify which column to embed (adjust if needed)
text_col = "full_text"  # or "text" / "clean_text" depending on your dataset

def average_vector(text, model):
    """Compute mean vector of all tokens that exist in model."""
    if not isinstance(text, str) or not text.strip():
        return None
    tokens = [w.lower() for w in text.split() if w.lower() in model.key_to_index]
    if not tokens:
        return None
    vecs = np.array([model[w] for w in tokens])
    return vecs.mean(axis=0)

# Compute embeddings and store as list
tqdm.pandas(desc="Computing embeddings")
df["embedding"] = df[text_col].progress_apply(lambda x: average_vector(x, model))

# (Optional) drop rows without embedding if you only want valid ones
# df = df[df["embedding"].notnull()]

df.head()


Computing embeddings: 100%|██████████| 4128/4128 [00:18<00:00, 217.67it/s]


,url,journal,year,title,full_text,citation,alternative.id,created,deposited,doi,...,Topic87,Topic88,Topic89,Topic90,Topic91,Topic92,Topic93,CSR,CSA,embedding
0,http://dx.doi.org/10.1016/j.pubrev.2003.09.003,Public Relations Review,2004,Exploring public relations in Croatia through ...,1 Introduction Public relations practices vary...,17,S0363811104000219,2004-05-01,2020-04-02,10.1016/j.pubrev.2003.09.003,...,0.0014,0.0001,0.0013,0.0002,0.0079,0.0151,0.0065,0.0018,0.0000,"[-0.1003513, 0.0858341, -0.0866577, -0.0038831..."
1,http://dx.doi.org/10.1016/j.pubrev.2003.11.001,Public Relations Review,2004,Cross-national conflict shifting: a global pub...,1 Introduction A maturation process of globali...,13,S0363811103000948,2004-01-27,2020-03-29,10.1016/j.pubrev.2003.11.001,...,0.0006,0.0258,0.0312,0.0150,0.0000,0.0007,0.0047,0.0835,0.0002,"[-0.11946913, 0.08105255, -0.09149462, 0.01086..."
2,http://dx.doi.org/10.1016/j.pubrev.2003.11.002,Public Relations Review,2004,Strategic public diplomacy and local press: ho...,As strategic public diplomacy events head of s...,32,S036381110300095X,2004-01-23,2019-02-15,10.1016/j.pubrev.2003.11.002,...,0.0002,0.0007,0.0002,0.0003,0.0000,0.0001,0.0443,0.0017,0.0000,"[-0.15308471, 0.12118629, -0.1334047, -0.01399..."
3,http://dx.doi.org/10.1016/j.pubrev.2003.11.003,Public Relations Review,2004,International agenda-building in cyberspace: a...,A 1970 New York Times magazine article argued ...,16,S0363811103000961,2004-01-22,2019-02-15,10.1016/j.pubrev.2003.11.003,...,0.0001,0.0002,0.0008,0.0011,0.0008,0.0052,0.0008,0.0014,0.0000,"[-0.13115636, 0.101520814, -0.13948408, -0.017..."
4,http://dx.doi.org/10.1016/j.pubrev.2003.11.004,Public Relations Review,2004,American Airlines’ use of mediated employee ch...,1 Introduction The September 11 2001 attacks h...,13,S0363811103000973,2004-01-22,2020-03-29,10.1016/j.pubrev.2003.11.004,...,0.0000,0.0083,0.0007,0.0040,0.0002,0.0564,0.0012,0.0094,0.0000,"[-0.12511124, 0.09210556, -0.1418865, 0.000818..."


In [4]:
df = df.drop(columns=["full_text"])
df

,url,journal,year,title,citation,alternative.id,created,deposited,doi,indexed,...,Topic87,Topic88,Topic89,Topic90,Topic91,Topic92,Topic93,CSR,CSA,embedding
0,http://dx.doi.org/10.1016/j.pubrev.2003.09.003,Public Relations Review,2004,Exploring public relations in Croatia through ...,17,S0363811104000219,2004-05-01,2020-04-02,10.1016/j.pubrev.2003.09.003,2024-09-14,...,0.0014,0.0001,0.0013,0.0002,0.0079,0.0151,0.0065,0.0018,0.00000,"[-0.1003513, 0.0858341, -0.0866577, -0.0038831..."
1,http://dx.doi.org/10.1016/j.pubrev.2003.11.001,Public Relations Review,2004,Cross-national conflict shifting: a global pub...,13,S0363811103000948,2004-01-27,2020-03-29,10.1016/j.pubrev.2003.11.001,2023-08-25,...,0.0006,0.0258,0.0312,0.0150,0.0000,0.0007,0.0047,0.0835,0.00020,"[-0.11946913, 0.08105255, -0.09149462, 0.01086..."
2,http://dx.doi.org/10.1016/j.pubrev.2003.11.002,Public Relations Review,2004,Strategic public diplomacy and local press: ho...,32,S036381110300095X,2004-01-23,2019-02-15,10.1016/j.pubrev.2003.11.002,2024-09-16,...,0.0002,0.0007,0.0002,0.0003,0.0000,0.0001,0.0443,0.0017,0.00000,"[-0.15308471, 0.12118629, -0.1334047, -0.01399..."
3,http://dx.doi.org/10.1016/j.pubrev.2003.11.003,Public Relations Review,2004,International agenda-building in cyberspace: a...,16,S0363811103000961,2004-01-22,2019-02-15,10.1016/j.pubrev.2003.11.003,2022-07-02,...,0.0001,0.0002,0.0008,0.0011,0.0008,0.0052,0.0008,0.0014,0.00000,"[-0.13115636, 0.101520814, -0.13948408, -0.017..."
4,http://dx.doi.org/10.1016/j.pubrev.2003.11.004,Public Relations Review,2004,American Airlines’ use of mediated employee ch...,13,S0363811103000973,2004-01-22,2020-03-29,10.1016/j.pubrev.2003.11.004,2023-08-19,...,0.0000,0.0083,0.0007,0.0040,0.0002,0.0564,0.0012,0.0094,0.00000,"[-0.12511124, 0.09210556, -0.1418865, 0.000818..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4123,https://doi.org/10.1207/s1532754xjprr1803_4,Journal of Public Relations Research,2006,First- and Second-Level Agenda-Building and Ag...,128,10.1207/s1532754xjprr1803_4,2006-08-25,2024-02-06,10.1207/s1532754xjprr1803_4,2024-09-03,...,0.0004,0.0001,0.0001,0.0002,0.0001,0.0002,0.0004,0.0004,0.00010,"[-0.14006828, 0.12637639, -0.14939941, -0.0313..."
4124,https://doi.org/10.1207/s1532754xjprr1804_1,Journal of Public Relations Research,2006,Toward Public Relations Theory-Based Study of ...,54,10.1207/s1532754xjprr1804_1,2006-09-07,2020-04-18,10.1207/s1532754xjprr1804_1,2024-05-28,...,0.0296,0.0001,0.0134,0.0003,0.0001,0.0702,0.0002,0.0056,0.00001,"[-0.09373342, 0.091842614, -0.09045223, -0.009..."
4125,https://doi.org/10.1207/s1532754xjprr1804_2,Journal of Public Relations Research,2006,How Activist Groups Use Websites in Media Rela...,96,10.1207/s1532754xjprr1804_2,2006-09-07,2019-04-21,10.1207/s1532754xjprr1804_2,2024-09-04,...,0.0003,0.0003,0.0004,0.0016,0.0001,0.0008,0.0003,0.0118,0.00000,"[-0.121581785, 0.07489445, -0.11334516, 0.0020..."
4126,https://doi.org/10.1207/s1532754xjprr1804_3,Journal of Public Relations Research,2006,Responding to Activism: An Experimental Analys...,29,10.1207/s1532754xjprr1804_3,2006-09-07,2019-01-13,10.1207/s1532754xjprr1804_3,2024-06-06,...,0.2571,0.0836,0.0005,0.0030,0.0004,0.0002,0.0002,0.0850,0.00020,"[-0.096751586, 0.081962965, -0.08126111, -0.01..."


In [6]:
import pandas as pd

topic_labels = pd.read_excel("01_Scripts_CSA_CSR/07_PREMBD/stm_93_done_4_plot.xlsx")
topic_labels = topic_labels.dropna(subset=["Topic"])
topic_labels = topic_labels.reset_index(drop=True)

topic_labels

,Topic,Label,Word Type,Top 20 Words,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,1.0,News Media,Prob,"news, medium, coverage, release, press, newspa...",news,medium,coverage,release,press,newspaper,...,time,positive,code,influence,tone,cover,content,percent,analyze,issue
1,2.0,Visual Communication,Prob,"image, visual, video, instagram, content, view...",image,visual,video,instagram,content,viewer,...,feature,impression,photo,attention,photograph,representation,focus,display,approach,platform
2,3.0,Emotional Appeal,Prob,"emotion, emotional, anger, negative, public, c...",emotion,emotional,anger,negative,public,cognitive,...,process,fear,behavior,affective,appraisal,feeling,effect,outrage,perceive,model
3,4.0,Networks,Prob,"network, organization, social, relationship, a...",network,organization,social,relationship,actor,capital,...,connect,centrality,share,resource,partner,public,type,node,individual,structural
4,5.0,SCCT,Prob,"crisis, organization, response, strategy, resp...",crisis,organization,response,strategy,responsibility,communication,...,attribution,scct,coomb,base,event,victim,blame,theory,level,preventable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,89.0,IMC,Prob,"market, communication, integrate, integration,...",market,communication,integrate,integration,customer,business,...,base,imc,activity,concept,advertise,process,management,term,create,cost
89,90.0,Legal Issues,Prob,"law, legal, court, speech, regulation, policy,...",law,legal,court,speech,regulation,policy,...,decision,issue,communication,federal,protect,lawyer,commercial,client,unite,supreme
90,91.0,Authenticity,Prob,"authenticity, authentic, message, silence, com...",authenticity,authentic,message,silence,communication,audience,...,express,behavior,speak,communicate,people,voice,cue,speech,manager,goal
91,92.0,Corporate Communication,Prob,"communication, department, management, organiz...",communication,department,management,organization,respondent,manager,...,survey,level,communicator,activity,strategic,internal,report,business,senior,team


In [11]:
# Start from Topic1 instead of Topic0
mapping = {
    f"Topic{i}": f"Topic {i}: {topic_labels.loc[i - 1, 'Label']}"
    for i in range(1, len(topic_labels) + 1)
}

mapping


{'Topic1': 'Topic 1: News Media',
 'Topic2': 'Topic 2: Visual Communication',
 'Topic3': 'Topic 3: Emotional Appeal',
 'Topic4': 'Topic 4: Networks',
 'Topic5': 'Topic 5: SCCT',
 'Topic6': 'Topic 6: Corporate Identity',
 'Topic7': 'Topic 7: Social Media',
 'Topic8': 'Topic 8: EOR',
 'Topic9': 'Topic 9: Issue Management',
 'Topic10': 'Topic 10: CSR Reporting',
 'Topic11': 'Topic 11: University',
 'Topic12': 'Topic 12: CHV',
 'Topic13': 'Topic 13: Feminism',
 'Topic14': 'Topic 14: Reputation',
 'Topic15': 'Topic 15: Discursive',
 'Topic16': 'Topic 16: BP',
 'Topic17': 'Topic 17: Listening',
 'Topic18': 'Topic 18: Nonprofit',
 'Topic19': 'Topic 19: Climate Change',
 'Topic20': 'Topic 20: Apologia',
 'Topic21': 'Topic 21: CSR Fit',
 'Topic22': 'Topic 22: Image Repair',
 'Topic23': 'Topic 23: Europe',
 'Topic24': 'Topic 24: Psychology',
 'Topic25': 'Topic 25: Ethics',
 'Topic26': 'Topic 26: Community Engagement',
 'Topic27': 'Topic 27: Gender',
 'Topic28': 'Topic 28: Identification',
 'Topi

In [12]:
# Rename columns in df
df = df.rename(columns=mapping)

df

,url,journal,year,title,citation,alternative.id,created,deposited,doi,indexed,...,Topic 87: STP/STOPs,Topic 88: Deviant Industry in CSR,Topic 89: IMC,Topic 90: Legal Issues,Topic 91: Authenticity,Topic 92: Corporate Communication,Topic 93: Sponsorship,CSR,CSA,embedding
0,http://dx.doi.org/10.1016/j.pubrev.2003.09.003,Public Relations Review,2004,Exploring public relations in Croatia through ...,17,S0363811104000219,2004-05-01,2020-04-02,10.1016/j.pubrev.2003.09.003,2024-09-14,...,0.0014,0.0001,0.0013,0.0002,0.0079,0.0151,0.0065,0.0018,0.00000,"[-0.1003513, 0.0858341, -0.0866577, -0.0038831..."
1,http://dx.doi.org/10.1016/j.pubrev.2003.11.001,Public Relations Review,2004,Cross-national conflict shifting: a global pub...,13,S0363811103000948,2004-01-27,2020-03-29,10.1016/j.pubrev.2003.11.001,2023-08-25,...,0.0006,0.0258,0.0312,0.0150,0.0000,0.0007,0.0047,0.0835,0.00020,"[-0.11946913, 0.08105255, -0.09149462, 0.01086..."
2,http://dx.doi.org/10.1016/j.pubrev.2003.11.002,Public Relations Review,2004,Strategic public diplomacy and local press: ho...,32,S036381110300095X,2004-01-23,2019-02-15,10.1016/j.pubrev.2003.11.002,2024-09-16,...,0.0002,0.0007,0.0002,0.0003,0.0000,0.0001,0.0443,0.0017,0.00000,"[-0.15308471, 0.12118629, -0.1334047, -0.01399..."
3,http://dx.doi.org/10.1016/j.pubrev.2003.11.003,Public Relations Review,2004,International agenda-building in cyberspace: a...,16,S0363811103000961,2004-01-22,2019-02-15,10.1016/j.pubrev.2003.11.003,2022-07-02,...,0.0001,0.0002,0.0008,0.0011,0.0008,0.0052,0.0008,0.0014,0.00000,"[-0.13115636, 0.101520814, -0.13948408, -0.017..."
4,http://dx.doi.org/10.1016/j.pubrev.2003.11.004,Public Relations Review,2004,American Airlines’ use of mediated employee ch...,13,S0363811103000973,2004-01-22,2020-03-29,10.1016/j.pubrev.2003.11.004,2023-08-19,...,0.0000,0.0083,0.0007,0.0040,0.0002,0.0564,0.0012,0.0094,0.00000,"[-0.12511124, 0.09210556, -0.1418865, 0.000818..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4123,https://doi.org/10.1207/s1532754xjprr1803_4,Journal of Public Relations Research,2006,First- and Second-Level Agenda-Building and Ag...,128,10.1207/s1532754xjprr1803_4,2006-08-25,2024-02-06,10.1207/s1532754xjprr1803_4,2024-09-03,...,0.0004,0.0001,0.0001,0.0002,0.0001,0.0002,0.0004,0.0004,0.00010,"[-0.14006828, 0.12637639, -0.14939941, -0.0313..."
4124,https://doi.org/10.1207/s1532754xjprr1804_1,Journal of Public Relations Research,2006,Toward Public Relations Theory-Based Study of ...,54,10.1207/s1532754xjprr1804_1,2006-09-07,2020-04-18,10.1207/s1532754xjprr1804_1,2024-05-28,...,0.0296,0.0001,0.0134,0.0003,0.0001,0.0702,0.0002,0.0056,0.00001,"[-0.09373342, 0.091842614, -0.09045223, -0.009..."
4125,https://doi.org/10.1207/s1532754xjprr1804_2,Journal of Public Relations Research,2006,How Activist Groups Use Websites in Media Rela...,96,10.1207/s1532754xjprr1804_2,2006-09-07,2019-04-21,10.1207/s1532754xjprr1804_2,2024-09-04,...,0.0003,0.0003,0.0004,0.0016,0.0001,0.0008,0.0003,0.0118,0.00000,"[-0.121581785, 0.07489445, -0.11334516, 0.0020..."
4126,https://doi.org/10.1207/s1532754xjprr1804_3,Journal of Public Relations Research,2006,Responding to Activism: An Experimental Analys...,29,10.1207/s1532754xjprr1804_3,2006-09-07,2019-01-13,10.1207/s1532754xjprr1804_3,2024-06-06,...,0.2571,0.0836,0.0005,0.0030,0.0004,0.0002,0.0002,0.0850,0.00020,"[-0.096751586, 0.081962965, -0.08126111, -0.01..."


In [14]:

df.to_json("01_Scripts_CSA_CSR/07_PREMBD/00_meta_w_STM.jsonl", orient="records", lines=True, force_ascii=False)


In [22]:
# topic_embedding_add.py
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

# --- Load model and STM topic file ---
model = Word2Vec.load("01_Scripts_CSA_CSR/07_PREMBD/models/word2vec_5gram_full.model")
stm_path = "01_Scripts_CSA_CSR/07_PREMBD/stm_93_done_4_plot.xlsx"

# --- Read topic file (each topic appears twice: Prob and FREX) ---
df_topic = pd.read_excel(stm_path)
df_topic["Topic"] = df_topic["Topic"].ffill().astype(int)
df_topic["Label"] = df_topic["Label"].fillna("").astype(str)

# --- Combine every 2 rows of keywords (Prob + FREX) ---
combined_rows = []
for i in range(0, len(df_topic), 2):
    if i + 1 < len(df_topic):
        topic_id = int(df_topic.iloc[i, 0])
        label = df_topic.iloc[i, 1]
        row_prob = df_topic.iloc[i, 4:]
        row_frex = df_topic.iloc[i + 1, 4:]
        words = pd.concat([row_prob, row_frex]).dropna().astype(str).str.lower().tolist()
        combined_rows.append({"Topic": topic_id, "Label": label, "Words": words})

df_combined = pd.DataFrame(combined_rows)

# --- Compute average embedding for each topic ---
def get_topic_vector(words, model):
    vecs = [model.wv[w] for w in words if w in model.wv]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

df_combined["Embedding"] = df_combined["Words"].apply(lambda w: get_topic_vector(w, model))

# --- Keep only Topic, Label, Embedding (as string) ---
df_export = df_combined[["Topic", "Label", "Embedding"]].copy()
df_export["Embedding"] = df_export["Embedding"].apply(lambda v: ",".join(map(str, v)))

# --- Save to new Excel ---
out_path = "01_Scripts_CSA_CSR/07_PREMBD/stm_93_done_4_plot_with_embeddings.xlsx"
df_export.to_excel(out_path, index=False)

# 2. Get the Keywords from the models

In [17]:
import os
import pandas as pd
from gensim.models import Word2Vec, KeyedVectors
from tqdm import tqdm

# ---------- CONFIG ----------
FULL_MODEL_PATH = "02_Output_CSA_CSR/02_WVmodel/word2vec_5gram.model"
YEARLY_DIR = "02_Output_CSA_CSR/02_WVmodel_by_year"
OUTPUT_DIR = "01_Scripts_CSA_CSR/07_PREMBD/keywords"
N_WORDS = None   # set to an integer (e.g., 50000) if you want to limit size

os.makedirs(OUTPUT_DIR, exist_ok=True)


def load_vectors(path):
    """Load either a Word2Vec or KeyedVectors file."""
    if path.endswith(".model"):
        model = Word2Vec.load(path).wv
    else:
        model = KeyedVectors.load(path, mmap='r')
    return model


def export_vocab(model_path, tag):
    """Save all words in the model vocabulary to CSV."""
    model = load_vectors(model_path)
    words = list(model.key_to_index.keys())
    if N_WORDS:
        words = words[:N_WORDS]
    df = pd.DataFrame({"word": words})
    out_path = os.path.join(OUTPUT_DIR, f"words_{tag}.csv")
    df.to_csv(out_path, index=False)
    print(f"Saved {len(df)} words → {out_path}")


if __name__ == "__main__":
    # 1. Full model
    export_vocab(FULL_MODEL_PATH, "full")

    # 2. Yearly models
    for fname in tqdm(sorted(os.listdir(YEARLY_DIR))):
        if fname.endswith(".model"):
            year = os.path.splitext(fname)[0].split("_")[-1]
            path = os.path.join(YEARLY_DIR, fname)
            export_vocab(path, year)


Saved 18220 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_full.csv


 29%|██▊       | 6/21 [00:00<00:00, 51.67it/s]

Saved 4103 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2004.csv
Saved 4473 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2005.csv
Saved 4333 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2006.csv
Saved 4596 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2007.csv
Saved 5002 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2008.csv
Saved 4905 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2009.csv
Saved 4831 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2010.csv
Saved 5047 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2011.csv
Saved 6581 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2012.csv
Saved 5721 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2013.csv


 81%|████████  | 17/21 [00:00<00:00, 40.26it/s]

Saved 6318 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2014.csv
Saved 6342 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2015.csv
Saved 6655 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2016.csv
Saved 6995 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2017.csv
Saved 6883 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2018.csv
Saved 6906 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2019.csv
Saved 6685 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2020.csv


100%|██████████| 21/21 [00:00<00:00, 39.85it/s]

Saved 7702 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2021.csv
Saved 7259 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2022.csv
Saved 7798 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2023.csv
Saved 7895 words → 01_Scripts_CSA_CSR/07_PREMBD/keywords/words_2024.csv
